In [ ]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("../src")

from config import load_config, get_abs_path
from log_config import logger
from feature_engineering import cria_features
import pandas as pd
import sys

logger.remove() # remove o logger default que foi importado do log_config
logger.add(sys.stderr, level="CRITICAL")

config = load_config()

paths = config["paths"]

# Todos os dados que entrarem na aplicação vão passar passar pelo pipeline de limpeza e construção das camadas do Data Lake.
# Então nós já vamos pegar da camada silves pra otimizar
df_prospects = pd.read_parquet(get_abs_path(paths["prospects_silver"]))
df_vagas = pd.read_parquet(get_abs_path(paths["vagas_silver"]))

## Pra começar vamos puxar os jsons que nós temos e vamos extrair uma pessoa aleatória para simular uma predição

In [2]:
um_prospect_aleatorio = df_prospects.sample(1)
um_prospect_aleatorio

,cod_vaga,titulo,nome,codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador
36229,8802,oracle pl sql senior,Eloah Cavalcante,21601,encaminhado requisitante,2022-03-30,30-03-2022,pretensao salarial r 17300 hrs pj r 20 mil clt...,Elisa Barbosa


## Desse prospect aleatório, nós vamos fazer as limpezas necessárias para chegar na tabela de predição

In [3]:
def clean_features_data(df):
    colunas_remover = ['analista_responsavel', 'cidade','cliente','cod_vaga','codigo', 'data_candidatura', 'data_final', 'data_inicial','data_requicisao',
                            'empresa_divisao','estado','limite_esperado_para_contratacao', 'local_trabalho','nome','recrutador',
                            'regiao','requisitante', 'situacao_candidado', 'solicitante_cliente', 'ultima_atualizacao', 'tipo_contratacao', 'nivel_academico',]
    colunas_remover_2 = ['titulo', 'comentario', 'titulo_vaga', 'prazo_contratacao','prioridade_vaga', 'nivel profissional', 'nivel_ingles',
                                'nivel_espanhol', 'areas_atuacao', 'principais_atividades','competencia_tecnicas_e_comportamentais', 'demais_observacoes',
                                'equipamentos_necessarios', 'habilidades_comportamentais_necessarias', 'valor_venda','valor_compra_1',]

    colunas_remover.extend(colunas_remover_2)

    df = df.drop(columns=colunas_remover)

    features = [
            col
            for col in df.columns
            if col != "target" and pd.api.types.is_numeric_dtype(df[col])
        ]
    X = df[features]

    return X

In [4]:
um_caso_aleatorio = um_prospect_aleatorio.merge(df_vagas, on="cod_vaga", how="left", suffixes=("", "_vaga"))

um_caso_aleatorio = cria_features(um_caso_aleatorio)

um_caso_aleatorio = clean_features_data(um_caso_aleatorio)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[String Similarity]: 100%|██████████| 1/1 [00:00<00:00, 640.84it/s]


[Embeddings] Gerando embeddings dos títulos...


Processing batches: 100%|██████████| 1/1 [00:00<00:00, 21.43it/s]


In [7]:
um_caso_aleatorio.values

array([[ 3.30000000e+01,  4.00000000e+00,  1.34119764e-05,
         5.10310307e-02, -1.39973611e-01,  1.53137624e-01,
         3.30000000e+01,  4.00000000e+00,  1.34119764e-05,
         5.10310307e-02, -1.39973611e-01,  1.53137624e-01,
         3.00000000e+01,  4.00000000e+00,  3.90899368e-05,
         5.10310195e-02, -1.74834356e-01,  1.58806533e-01,
         9.40000000e+01,  1.60000000e+01, -1.60311665e-05,
         5.10310382e-02, -1.86851099e-01,  1.43074393e-01,
         1.00000000e+00,  1.00000000e+00]])

## Teste com a inferência na API

In [2]:
import pandas as pd
from src.config import load_config, get_abs_path

config = load_config()
paths = config["paths"]

df = pd.read_parquet(get_abs_path(paths["dataset_features"]))
df

,target,nivel_acad_desconhecido,nivel_acad_doutorado completo,nivel_acad_doutorado cursando,nivel_acad_ensino fundamental completo,nivel_acad_ensino medio completo,nivel_acad_ensino medio incompleto,nivel_acad_ensino superior completo,nivel_acad_ensino superior cursando,nivel_acad_ensino superior incompleto,...,demais_observacoes_emb_min,demais_observacoes_emb_max,comentario_nchar,comentario_nwords,comentario_emb_mean,comentario_emb_std,comentario_emb_min,comentario_emb_max,titulo_sim_ratio,sim_titulo_vs_vaga
0,0,False,False,False,False,False,False,True,False,False,...,-0.161850,0.117181,26,5,-0.000235,0.051030,-0.176741,0.204723,1.000000,1.000000
1,0,False,False,False,False,False,False,True,False,False,...,-0.161850,0.117181,48,8,0.000284,0.051030,-0.155262,0.165039,1.000000,1.000000
2,1,False,False,False,False,True,False,False,False,False,...,-0.185271,0.210202,22,5,0.000357,0.051030,-0.165430,0.155121,1.000000,1.000000
3,0,False,False,False,False,True,False,False,False,False,...,-0.185271,0.210202,12,1,0.000110,0.051031,-0.177052,0.497576,1.000000,1.000000
4,0,False,False,False,False,False,False,False,False,False,...,-0.153048,0.165421,12,1,0.000110,0.051031,-0.177052,0.497576,0.066667,0.188192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56697,0,False,False,False,False,False,False,True,False,False,...,-0.129788,0.161187,12,1,0.000110,0.051031,-0.177052,0.497576,0.250000,0.124157
56698,0,False,False,False,False,False,False,True,False,False,...,-0.177052,0.497576,58,7,0.000314,0.051030,-0.126359,0.117563,1.000000,1.000000
56699,0,False,False,False,False,False,False,True,False,False,...,-0.177052,0.497576,12,1,0.000110,0.051031,-0.177052,0.497576,1.000000,1.000000
56700,1,False,False,False,False,False,False,True,False,False,...,-0.177052,0.497576,12,1,0.000110,0.051031,-0.177052,0.497576,1.000000,1.000000


In [3]:
df = df.drop(columns=["target"])

caso_de_uma_pessoa_aleatoria = df.sample(1)
caso_de_uma_pessoa_aleatoria

,nivel_acad_desconhecido,nivel_acad_doutorado completo,nivel_acad_doutorado cursando,nivel_acad_ensino fundamental completo,nivel_acad_ensino medio completo,nivel_acad_ensino medio incompleto,nivel_acad_ensino superior completo,nivel_acad_ensino superior cursando,nivel_acad_ensino superior incompleto,nivel_acad_ensino tecnico completo,...,demais_observacoes_emb_min,demais_observacoes_emb_max,comentario_nchar,comentario_nwords,comentario_emb_mean,comentario_emb_std,comentario_emb_min,comentario_emb_max,titulo_sim_ratio,sim_titulo_vs_vaga
46142,False,False,False,False,False,False,True,False,False,False,...,-0.141665,0.136526,12,1,0.00011,0.051031,-0.177052,0.497576,1.0,1.0


In [4]:
random_row_df = df.sample(n=1)
payload_dict = random_row_df.iloc[0].to_dict()

In [5]:
import json
json.dumps(payload_dict)

'{"nivel_acad_desconhecido": false, "nivel_acad_doutorado completo": false, "nivel_acad_doutorado cursando": false, "nivel_acad_ensino fundamental completo": false, "nivel_acad_ensino medio completo": false, "nivel_acad_ensino medio incompleto": false, "nivel_acad_ensino superior completo": true, "nivel_acad_ensino superior cursando": false, "nivel_acad_ensino superior incompleto": false, "nivel_acad_ensino tecnico completo": false, "nivel_acad_ensino tecnico cursando": false, "nivel_acad_ensino tecnico incompleto": false, "nivel_acad_mestrado completo": false, "nivel_acad_mestrado cursando": false, "nivel_acad_pos graduacao completo": false, "nivel_acad_pos graduacao cursando": false, "nivel_acad_pos graduacao incompleto": false, "nivel_acad_nan": false, "tipo_contr_candidato podera escolher hunting pj autonomo": false, "tipo_contr_candidato podera escolher pj autonomo": false, "tipo_contr_clt cotas": false, "tipo_contr_clt cotas clt full": false, "tipo_contr_clt cotas cooperado": fal

In [6]:
payload_dict

{'nivel_acad_desconhecido': False,
 'nivel_acad_doutorado completo': False,
 'nivel_acad_doutorado cursando': False,
 'nivel_acad_ensino fundamental completo': False,
 'nivel_acad_ensino medio completo': False,
 'nivel_acad_ensino medio incompleto': False,
 'nivel_acad_ensino superior completo': True,
 'nivel_acad_ensino superior cursando': False,
 'nivel_acad_ensino superior incompleto': False,
 'nivel_acad_ensino tecnico completo': False,
 'nivel_acad_ensino tecnico cursando': False,
 'nivel_acad_ensino tecnico incompleto': False,
 'nivel_acad_mestrado completo': False,
 'nivel_acad_mestrado cursando': False,
 'nivel_acad_pos graduacao completo': False,
 'nivel_acad_pos graduacao cursando': False,
 'nivel_acad_pos graduacao incompleto': False,
 'nivel_acad_nan': False,
 'tipo_contr_candidato podera escolher hunting pj autonomo': False,
 'tipo_contr_candidato podera escolher pj autonomo': False,
 'tipo_contr_clt cotas': False,
 'tipo_contr_clt cotas clt full': False,
 'tipo_contr_clt 

In [7]:
import requests

api_url = "http://127.0.0.1:3000/predict" # Endpoint definido no service.py

response = requests.post(
    api_url,
    headers={"Content-Type": "application/json"},
    json=payload_dict  # 'requests' converte o dict para JSON automaticamente
)

# Verificar se a requisição foi bem-sucedida (código 2xx)
#response.raise_for_status() # Levanta um erro para códigos HTTP ruins (4xx ou 5xx)

In [9]:
response.content

b'{"error":"Not able to process the request in 60 seconds"}'

In [ ]:
# Imprimir a resposta da API
print(f"Status Code: {response.status_code}")
print("Resposta da API:")
print(response.json()) # Converte a resposta JSON da API em um dict Python

In [7]:
import pickle

with open(paths["modelo_treinado"], "rb") as f:
    dados_carregados = pickle.load(f)

modelo = dados_carregados["model"]
imputer = dados_carregados["imputer"]
scaler = dados_carregados["scaler"]
features = dados_carregados["features"]

In [15]:
caso_de_uma_pessoa_aleatoria.values[0]

array([False, False, False, False, False, False, True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, 601, 66,
       -0.00010113581083714962, 0.05103093758225441, -0.14400725066661835,
       0.1506543755531311, 533, 59, 0.0001684475428191945,
       0.051030758768320084, -0.1545673906803131, 0.19803106784820557,
       149, 21, 0.00018128710507880896, 0.0510307140648365,
       -0.14166459441184998, 0.13652615249156952, 12, 1,
       0.00011021317914128304, 0.05103091895580292, -0.1770523190498352,
       0.49757611751556396, 1.0, 0.9999998807907104], dtype=object)

In [13]:
caso_imputer = imputer.transform([caso_de_uma_pessoa_aleatoria.values[0]])[0]
caso_scaler = scaler.transform([caso_imputer])[0]
modelo.predict_proba([caso_scaler])

/home/felip/projetos/datathon-mle/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([[0.91247835, 0.08752165]])

In [33]:
pd.DataFrame(caso_scaler).to_dict(orient="split")

{'index': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84],
 'columns': [0],
 'data': [[-0.0536109596622732],
  [0.0],
  [0.0],
  [-0.011501726586678177],
  [-0.34871656695933395],
  [-0.0269819722534416],
  [0.5493574952857494],
  [-0.190343703073752],
  [-0.10758239089608226],
  [-0.2658824563999154],
  [-0.020470406369719855],
  [-0.026569716472705562],
  [-0.011501726586678179],
  [-0.013281342715995811],
  [-0.06604453909502439],
  [-0.034842032732843226],
  [-0.01626697374183083],
  [0.0],
  [-0.011501726586678179],
  [-0.0115017265866781

In [29]:
caso_scaler

array([-0.05361096,  0.        ,  0.        , -0.01150173, -0.34871657,
       -0.02698197,  0.5493575 , -0.1903437 , -0.10758239, -0.26588246,
       -0.02047041, -0.02656972, -0.01150173, -0.01328134, -0.06604454,
       -0.03484203, -0.01626697,  0.        , -0.01150173, -0.01150173,
       -0.15174522, -0.02698197, -0.01626697, -0.03903137, -0.02529278,
       -0.0322057 , -0.46855697, -0.02047041, -0.00813268, -0.02348272,
       -0.00939091, -0.03354965, -0.05150205, -0.02047041, -0.08402333,
       -0.01242342,  2.66298875, -0.23447938, -0.00813268, -0.01626697,
       -0.11742895, -0.19593198, -0.01484932, -0.35212691, -0.00664023,
       -0.01328134, -0.00664023, -0.01150173, -0.71257375, -0.05319579,
       -0.02895559, -0.31454936, -0.10199157, -0.02615098, -0.00664023,
       -0.03484203, -0.01049948, -0.01408715,  0.        , -0.23281937,
       -0.1821328 , -0.67181586,  0.05823488,  0.20194439,  0.14673566,
       -0.70014354, -0.69343643,  0.32598417,  0.18023588,  0.38